# store geostrophic fields

Xiaolong initial [notebook](https://github.com/xy6g13/equinox_working/blob/master/sandbox/Reconstruction/Geostrophy%20metric.ipynb)

In [1]:
import os, sys, shutil
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

import xgcm

from mitequinox.utils import *
from mitequinox.dynamic import *
from mitequinox.plot import *

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=12, walltime='12:00:00')
w = cluster.scale(60*12)
#w = cluster.scale(5) # postprocessing

/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
cluster

---
# load data


In [5]:
grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])

dsU = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU'))
dsV = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV'))
dsE = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('Eta'))

ds = (xr.merge([dsU,dsV, dsE], join='inner')
      .assign_coords(**grd.variables))
grid = xgcm.Grid(ds, periodic=['X', 'Y'])

# add coriolis parameters to dataset
#omega = 7.3/100000 # XY, 
# see, http://mitgcm.org/public/r2_manual/final/code_reference/vdb/names/R.htm
f = 2*omega_earth*np.sin(np.deg2rad(ds['YG'])) # at vorticity points
f_i = grid.interp(f,'X').chunk({'i':None}) # at v points
f_j = grid.interp(f,'Y').chunk({'j':None}) # at u points
ds0 = ds.assign_coords(f=f,f_i=f_i,f_j=f_j)

---

## start computing and storing

is the storing really necessary?

In [6]:
terms = ['u_coriolis_linear','u_gradp', 'v_coriolis_linear','v_gradp']

In [7]:
#Nt, Ni, Nj = 24*20, 188, 96
Nt, Ni, Nj = 24*40, 376, 188

In [9]:
overwrite=True

#face=1
#for face in range(13):
for face in range(11,12):

    print('--- start processing face %d'%face)
    ds = ds0.sel(face=face)

    #t = terms[1]
    for t in terms:
        
        bterm = get_mbal(t, ds, grid)

        if t[0]=='u':
            chunks = {'time': Nt, 'i_g': Ni, 'j': Nj}
        elif t[0]=='v':
            chunks = {'time': Nt, 'i': Ni, 'j_g': Nj}
        bterm = bterm.isel(time=slice(len(ds.time)//Nt *Nt))
        bterm = bterm.chunk(chunks)

        file_out = work_data_dir+'mbal/%s_f%02d.zarr'%(t,face)
        if not os.path.isdir(file_out) or overwrite:
            bterm.to_dataset().to_zarr(file_out, mode='w')
        print('  %s, face=%d - processed'%(t, face))

print('mbal terms computed for face=%d'%(face))

--- start processing face 11
  u_gradp, face=11 - processed
mbal terms computed for face=11


In [10]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client


---

In [17]:
w = cluster.scale_up(30)

In [ ]:
client.restart()

In [9]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
